# Build an autonomous agent that can browse the web (from scratch)!

**This demo is powered by Solar LLM https://www.upstage.ai/**

To get started, make sure you've followed the steps in the README.

In [1]:
from llm import call_solar
from termcolor import cprint


def llm(prompt):
    cprint(prompt, "black")
    cprint(call_solar(prompt), "green")

print(call_solar("Hello world!"))

/Users/eric/dev/order-a-pizza/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Hello! How can I assist you today?


# Our Goal: Build an autonomous agent that can browse the web

The agent should be able to do general things like:
- "Generate a list of leads and contact info for my startup"
- "Wish my friend Onome a happy birthday on Instagram"
- "Generate a competitive analysis market landscape report about AI"

### Task: "Order a large pepperoni pizza from Domino's for me"


In [2]:
task = "Order me a large pepperoni pizza from Domino's for delivery to 275 Brannan St, San Francisco, CA 94107"

currently_open_url = "https://www.dominos.com/"

prompt = f"""\
You are a helpful assistant who can interact with a browser on behalf of the user. Your purpose is to help a user complete the following task:
{task}

The website {currently_open_url} is open.

What is the most logical next action to take?
"""

llm(prompt)

You are a helpful assistant who can interact with a browser on behalf of the user. Your purpose is to help a user complete the following task:
Order me a large pepperoni pizza from Domino's for delivery to 275 Brannan St, San Francisco, CA 94107

The website https://www.dominos.com/ is open.

What is the most logical next action to take?

The most logical next action to take would be to navigate to the "Order" or "Online Ordering" section of the Domino's website. You can do this by clicking on the "Order" button in the top right corner of the page, or by clicking on the "Order Online" link in the main navigation bar. Once you are in the ordering section, you can select the size and type of pizza you want, choose the "275 Brannan St, San Francisco, CA 94107" as the delivery address, and complete the order by providing your payment information.


# Step 1: Give the agent eyes!
To order a pizza, the agent needs to know what the webpage looks like. So let's pass the HTML to the LLM so it can "see" the webpage.

- This is RAG (retrieval augmented generation)!
- Alternatively we could pass an image, but that's expensive and comes with a whole other set of challenges (and doesn't work on non-multimodal LLMs)

In [3]:
fake_dominos_html = """<html>
    <body>
        <header>
            <h1>Welcome to Domino's Pizza</h1>
            <p>Order online for delivery or pickup</p>
            <a href="#" id="order-button">Order Now</a>
        </header>

        <main>
            <h2>Our Specialties</h2>
            <p>Discover our delicious range of pizzas, sides, and desserts.</p>
        </main>
    </body>
</html>"""

prompt = f"""\
You are a helpful assistant who can interact with a browser on behalf of the user. Your purpose is to help a user complete the following task:
{task}

You must carefully read over the current webpage's HTML, and based on the current state of the webpage, decide what is the most logical next action to take.

The webpage {currently_open_url} is open. Carefully analyze the HTML, and based on the HTML contents, determine the next action to take to help the user get closer to achieving their task. The HTML of the current webpage is:

{fake_dominos_html}
"""

llm(prompt)

You are a helpful assistant who can interact with a browser on behalf of the user. Your purpose is to help a user complete the following task:
Order me a large pepperoni pizza from Domino's for delivery to 275 Brannan St, San Francisco, CA 94107

You must carefully read over the current webpage's HTML, and based on the current state of the webpage, decide what is the most logical next action to take.

The webpage https://www.dominos.com/ is open. Carefully analyze the HTML, and based on the HTML contents, determine the next action to take to help the user get closer to achieving their task. The HTML of the current webpage is:

<html>
    <body>
        <header>
            <h1>Welcome to Domino's Pizza</h1>
            <p>Order online for delivery or pickup</p>
            <a href="#" id="order-button">Order Now</a>
        </header>

        <main>
            <h2>Our Specialties</h2>
            <p>Discover our delicious range of pizzas, sides, and desserts.</p>
        </main>
    <

# Step 2: Tell the agent what actions it can perform
The agent should be able to click on buttons, go to a url, and fill out form fields

- Basically, we're giving the llm a mouse and a keyboard
- This is the equivalent of LangChain tools!

In [4]:
actions_agent_can_perform = """\
click_html_element(id: 'str'): Click an HTML <a> tag or <button> identified by its ID

fill_text_in_input(id: 'str', text: 'str'): Type text into an input or textarea identified by its ID. Important: This function can ONLY be called with an ID that belongs directly to an <input> or <textarea> tag.

choose_dropdown_values(id: 'str', values: 'List[str]'): Select value(s) for a <select> tag identified by its ID. The `values` list must contain at least one string option to select. Important: This function can ONLY be called with an ID that belongs directly to a <select> tag.

go_to_url(url: 'str'): Open a webpage by URL"""

In [5]:

prompt = f"""\
You are a helpful assistant who can interact with a browser on behalf of the user. Your purpose is to help a user complete the following task:
{task}

You can perform the following actions to interact with the browser:
{actions_agent_can_perform}

You must carefully read over the current webpage's HTML, and based on the current state of the webpage, decide the single most logical next action to take to help advance in achieving the task.

The webpage {currently_open_url} is open. Carefully analyze the HTML, and based on the HTML contents, determine the next action to take to help the user get closer to achieving their task. The HTML of the current webpage is:

{fake_dominos_html}
"""

llm(prompt)

You are a helpful assistant who can interact with a browser on behalf of the user. Your purpose is to help a user complete the following task:
Order me a large pepperoni pizza from Domino's for delivery to 275 Brannan St, San Francisco, CA 94107

You can perform the following actions to interact with the browser:
click_html_element(id: 'str'): Click an HTML <a> tag or <button> identified by its ID

fill_text_in_input(id: 'str', text: 'str'): Type text into an input or textarea identified by its ID. Important: This function can ONLY be called with an ID that belongs directly to an <input> or <textarea> tag.

choose_dropdown_values(id: 'str', values: 'List[str]'): Select value(s) for a <select> tag identified by its ID. The `values` list must contain at least one string option to select. Important: This function can ONLY be called with an ID that belongs directly to a <select> tag.

go_to_url(url: 'str'): Open a webpage by URL

You must carefully read over the current webpage's HTML, and

# Step 3: Tell the agent to return a JSON
We want the agent to output the response in a structured format so we can parse the action and execute it outside of the LLM!

- Alternatively you can use the ChatGPT function calling API / structured output API

In [6]:
json_structure = """\
{
    "action": "Call the action in the following format: action_name(param_name='argument')"
}
"""

In [7]:

prompt = f"""\
You are a helpful assistant who can interact with a browser on behalf of the user. Your purpose is to help a user complete the following task:
{task}

You can perform the following actions to interact with the browser:
{actions_agent_can_perform}

Your response must be in JSON format with a field named "action":
{json_structure}

You must carefully read over the current webpage's HTML, and based on the current state of the webpage, decide the single most logical next action to take to help advance in achieving the task.

The webpage {currently_open_url} is open. Carefully analyze the HTML, and based on the HTML contents, determine the next action to take to help the user get closer to achieving their task. The HTML of the current webpage is:

{fake_dominos_html}
"""

llm(prompt)

You are a helpful assistant who can interact with a browser on behalf of the user. Your purpose is to help a user complete the following task:
Order me a large pepperoni pizza from Domino's for delivery to 275 Brannan St, San Francisco, CA 94107

You can perform the following actions to interact with the browser:
click_html_element(id: 'str'): Click an HTML <a> tag or <button> identified by its ID

fill_text_in_input(id: 'str', text: 'str'): Type text into an input or textarea identified by its ID. Important: This function can ONLY be called with an ID that belongs directly to an <input> or <textarea> tag.

choose_dropdown_values(id: 'str', values: 'List[str]'): Select value(s) for a <select> tag identified by its ID. The `values` list must contain at least one string option to select. Important: This function can ONLY be called with an ID that belongs directly to a <select> tag.

go_to_url(url: 'str'): Open a webpage by URL

Your response must be in JSON format with a field named "act

# Step 4: Make the agent think before taking an action

LLMs are prone to hallucination. If you ask the agent to explicitly describe what it observes in the HTML and to **reason** out loud, it is much more likely to arrive at a logical decision. Without this observation/reasoning step, the agent might hallucinate and:

- Make up HTML IDs that don't exist
- Choose an incorrect action that doesn't advance the task

In [8]:
better_json_structure = """\
{
    "observations": "Carefully read the HTML of the current webpage. Based on the HTML, explain the purpose of the webpage and identify the important HTML elements. Pay close attention to the state of informational elements such as the cart, input validation errors, sign in status, etc.",
    "reasoning": "Think critically about what action you can perform on the HTML to help advance in the user task. Describe why you think this is the best action to perform given the contents of the current page.",
    "action": "Call the action in the following format: action_name(param_name='argument')"
}"""

In [9]:

prompt = f"""\
You are a helpful assistant who can interact with a browser on behalf of the user. Your purpose is to help a user complete the following task:
{task}

You can perform the following actions to interact with the browser:
{actions_agent_can_perform}

Your response must be in JSON format with fields named "observations", "reasoning", and "action":
{better_json_structure}

You must carefully read over the current webpage's HTML, and based on the current state of the webpage, decide the single most logical next action to take to help advance in achieving the task.

The webpage {currently_open_url} is open. Carefully analyze the HTML, and based on the HTML contents, determine the next action to take to help the user get closer to achieving their task. The HTML of the current webpage is:

{fake_dominos_html}
"""

llm(prompt)

You are a helpful assistant who can interact with a browser on behalf of the user. Your purpose is to help a user complete the following task:
Order me a large pepperoni pizza from Domino's for delivery to 275 Brannan St, San Francisco, CA 94107

You can perform the following actions to interact with the browser:
click_html_element(id: 'str'): Click an HTML <a> tag or <button> identified by its ID

fill_text_in_input(id: 'str', text: 'str'): Type text into an input or textarea identified by its ID. Important: This function can ONLY be called with an ID that belongs directly to an <input> or <textarea> tag.

choose_dropdown_values(id: 'str', values: 'List[str]'): Select value(s) for a <select> tag identified by its ID. The `values` list must contain at least one string option to select. Important: This function can ONLY be called with an ID that belongs directly to a <select> tag.

go_to_url(url: 'str'): Open a webpage by URL

Your response must be in JSON format with fields named "obse

# Give the agent memory!

- The agent is just an LLM running in a while loop
- It needs to have context about what actions it has already performed!

In [10]:
fake_dominos_html_with_item_in_cart = """<html>
    <body>
        <header>
            <h1>Welcome to Domino's Pizza</h1>
            <p>Order online for delivery or pickup</p>
            <a href="#" id="order-button">Order Now</a>
            <div id = "cart">1 item in cart</div>
        </header>

        <main>
            <h2>Our Specialties</h2>
            <p>Discover our delicious range of pizzas, sides, and desserts.</p>
        </main>
    </body>
</html>"""

In [11]:
previous_actions_taken = "You have already entered your delivery address and added a pizza to the cart. There is now a large pepperoni pizza in the user's cart. You are ready to checkout."

prompt = f"""\
You are a helpful assistant who can interact with a browser on behalf of the user. Your purpose is to help a user complete the following task:
{task}

You can perform the following actions to interact with the browser:
{actions_agent_can_perform}

Important: You have already performed the following actions: {previous_actions_taken}

Your response must be in JSON format with fields named "observations", "reasoning", and "action":
{better_json_structure}

You must carefully read over the current webpage's HTML, and based on the current state of the webpage, decide the single most logical next action to take to help advance in achieving the task.

The webpage {currently_open_url} is open. Carefully analyze the HTML, and based on the HTML contents, determine the next action to take to help the user get closer to achieving their task. The HTML of the current webpage is:

{fake_dominos_html_with_item_in_cart}
"""

llm(prompt)

You are a helpful assistant who can interact with a browser on behalf of the user. Your purpose is to help a user complete the following task:
Order me a large pepperoni pizza from Domino's for delivery to 275 Brannan St, San Francisco, CA 94107

You can perform the following actions to interact with the browser:
click_html_element(id: 'str'): Click an HTML <a> tag or <button> identified by its ID

fill_text_in_input(id: 'str', text: 'str'): Type text into an input or textarea identified by its ID. Important: This function can ONLY be called with an ID that belongs directly to an <input> or <textarea> tag.

choose_dropdown_values(id: 'str', values: 'List[str]'): Select value(s) for a <select> tag identified by its ID. The `values` list must contain at least one string option to select. Important: This function can ONLY be called with an ID that belongs directly to a <select> tag.

go_to_url(url: 'str'): Open a webpage by URL

Important: You have already performed the following actions: